# Urban Sounds Classification

## Kullanacağımız Kütüphaneler

* Kodu Colab'da yazdığımız için kullanacağımız ses, veri gibi materyalleri Google Drive'a atmak gerekiyor.

* drive.mount: Aynı Linux'ta olduğu gibi Drive'da da kök dizin vardır. Drive'da kök dizin /content/drive şeklindedir. Bu kod ile kök dizine bağlanılır.

* os.chdir: Linux'ta da olduğu gibi chdir komutu bizi istediğimiz klasöre atar. Dikkat edersen burada da /content/drive dosya yolu var. Ama ben My Drive dosyasına bağlanmak istiyorum. Bu dosya tıpkı Windows'ta Desktop gibi bir klasör. Yani Desktop'ın Google Drive'da karşılığı My Drive'dır.  UrbanSound8K.tar.gz(sıkıştırılmış dosya) dosyasını da My Drive içinde global_ai_hub diye daha önce oluşturduğum klasöre yüklemiştim. Bu yüzden chdir ile dosya yolunu buraya getirdim. Artık bu klasörün içindeyim.

In [1]:
from google.colab import drive
import os
drive.mount("/content/drive")
os.chdir("/content/drive/My Drive/global_ai_hub")

Mounted at /content/drive


## Diğer Kütüphaneler

* librosa: Bu kütüphane ses dosyalarını(mp3, wav gibi dosyalar) okumamızı sağlar.

* librosa.display: librosa'nın bir özelliği de ses dosyalarını resme çevirebilmesidir. Bunun için librosa'nın display metodunu kullanılır.

* cv2: OpenCV kütüphanesi, resimler üzerinde sonsuz sayıda işlemler yapmamızı sağlar. Resimleri okur, gösterir, resim üzerinde değişiklik yaparsak kaydeder. Küçültme, büyütme, renkli resimleri gri formata dönüştürme gibi daha bir sürü işlem yapmamız sağlar.

In [2]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

* os.chdir ile global_ai_hub içine girmiştim. Daha önce global_ai_hub'ın içine UrbanSound8K.tar.gz sıkıştırılmış dosyasını yüklemiştim. Aşağıdaki komutla bu sıkıştırılmış dosyayı açtım.

In [3]:
#Unzip  UrbanSound8K.tar.gz
!tar -xvf UrbanSound8K.tar.gz

## 0. Resim Dosyaları Oluşturma

* Sıkıştırılmış dosyayı açınca içinde sound ve metadata adında iki klasör olan bir klasör gelmiş olacak. sound klasöründe ses dosyaları var, metadata içinde de bir tablo dosyası var(UrbanSound8K.csv). Bu dosyayı pandas kütüphanesi vasıtasıyla açalım.


* En soldaki sütunda ses dosyalarının isimleri var. Dikkat edersen dosya isminde ilk "-" ifadesinden sonraki numara sınıf/etiket'lerdir. classID sütunundaki etiketlerle bu numaraların aynı olduğunu aşağıdaki tablodan görebilirsin. Dolayısıyla ileride bu dosyaların etiketlerine dosya isimlerinden(100032-'3'-0-0.wav gibi) erişeceğiz.

* Toplam 10 adet etiket var. Detayları da şöyle:

0 = air_conditioner

1 = car_horn

2 = children_playing

3 = dog_bark

4 = drilling

5 = engine_idling

6 = gun_shot

7 = jackhammer

8 = siren

9 = street_music

In [4]:
import pandas as pd
df=pd.read_csv("/content/drive/My Drive/global_ai_hub/UrbanSound8K/metadata/UrbanSound8K.csv")
df.head(15)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
5,100263-2-0-143.wav,100263,71.500000,75.500000,1,5,2,children_playing
6,100263-2-0-161.wav,100263,80.500000,84.500000,1,5,2,children_playing
7,100263-2-0-3.wav,100263,1.500000,5.500000,1,5,2,children_playing
8,100263-2-0-36.wav,100263,18.000000,22.000000,1,5,2,children_playing
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn


* İlk amacımız, ses dosyalarını resimlere dönüştürüp etiketlerine göre bir klasöre atmak. Yani her etiket için bir klasör olacak(toplam 10 etiket olduğu için 10 klasör olacak) ve her klasör bir etiketin resimlerini tutacak. Mesela '8' adında klasör, 8(siren) etiketli ses dosyalarının resimlerini tutsun istiyoruz. 

* Bunun için evvela bu 10 klasörü içeren "our_spectrograms" adında bir klasör oluşturalım. Daha sonra bunun içine 0'dan 9'a klasör oluşturalım. 

In [5]:
os.mkdir("our_spectrograms") #Ana Klasör

In [6]:
#Toplam 10 Sınıf var
num_of_classes=10 
#0'dan 9'a isimlendirilmiş 10 klasörü bu dosya yoluna atacağız.
main_folder= "/content/drive/My Drive/global_ai_hub/our_spectrograms" 

* Aşağıdaki komutu çalıştırdığımızda 0, 1, 2, 3, ... , 8, 9 isimlerinde 10 klasör oluşmuş olacak.

In [7]:
os.mkdir(main_folder)
for i in range(num_of_classes):
  os.mkdir(main_folder+"/"+str(i))

## 1. Spectrogram Oluşturma 

* UrbanSound8K.tar.gz'den çıkarttığımız ses dosyalarını resime dönüştürelim.

* Aşağıdaki fonksiyon, proje dökümanında vardı. Aynen aldım. Bu fonksiyona bir ses dosyası verirsen onu ham resme dönüştürüyor. Dolayısıyla elimizdeki tüm ses dosyalarını bu fonksiyona vermemiz gerekiyor. 

In [8]:
def create_spectrogram(y):
  spec= librosa.feature.melspectrogram(y=y)
  spec_conv=librosa.amplitude_to_db(spec, ref=np.max)
  return spec_conv #Ham Resim

* Aşağıdaki fonksiyon ise ham resmi gerçek renkli resme dönüştürecek ve kaydedecek.

* Bu foknsiyon iki parametre alıyor:

=> SOUND_PATH: Ses dosyasının dosya yolu(/content/drive/My Drive/global_ai_hub/UrbanSound8K/sound/fold1/7061-6-0-0.wav gibi)

=> IMAGE_PATH: Sesi resme dönüştürdükten sonra resmi atacağımız dosya yolu(/content/drive/My Drive/global_ai_hub/our_spectrograms/6 gibi)

In [9]:
def save_image(SOUND_PATH, IMAGE_PATH):
  y, sr = librosa.load(SOUND_PATH)
  spec_conv=create_spectrogram(y)
  librosa.display.specshow(spec_conv, sr=sr)
  plt.savefig(IMAGE_PATH, bbox_inches='tight', pad_inches=0, transparent=True) #Resmi IMAGE_PATH'e kaydediyor.
  plt.close()

### NOT: UrbanSound8K.tar.gz sıkıştırılmış dosyasını çıkardğımızda ses dosyalarının ayrı ayrı 10 fold ile tutulduğunu görüyoruz. Dolayısıyla tüm ses dosyalarını resme dönüştürebilmek için sırasıyla tüm 10 fold'u da gezip tüm ses dosyalarını okumamız lazım.

### Aşağıdaki kod bloğunda range(1,6) ile dönen dış for'un aslında range(1,11) ile dönmesi gerekirdi. Yani ilk 5 fold'u değil, 10 fold'un hepsini okuması gerekirdi. Ancak sadece 5 ses fold'undan veri alabiliyorum. Çünkü daha fazla fold'dan veri almaya kalkıştığımda GPU yeterli gelmiyor ve "OUT OF MEMORY" hatasıyla karşılaşıyorum.

### Bu yüzden sadece ses verilerinin nasıl resme dönüştürüldüğünü göstermek amacıyla bu kod bloklarını yazdık. Ancak bize tüm ses dosyalarının resim hali lazım olduğu için Global AI Hub'ın bize sağladığı ses resimlerini doğrudan eğitim için kullanacağız.

In [10]:
# fold1, fold2, fold3..., fold5 için 1,2,3,4,5 sayılarını dönsün:
for folder_index in range(1,6):
  #/content/drive/My Drive/global_ai_hub/UrbanSound8K/audio/fold1 gibi dosya yolları oluşturalım:
  sound_folder="/content/drive/My Drive/global_ai_hub/UrbanSound8K/audio/fold"+str(folder_index)
  #os.listdir linux'taki ls komutu ile aynı. İçindeki dosya isimlerinin hepsini liste yapıp bize veriyor.
  for sound_file in os.listdir(sound_folder):
    if sound_file.endswith(".wav"): #Sadece .wav uzantılı dosyaları alsın.
        sound_label=sound_file.split("-")[1] # Ses dosyasının etiketi. 3(dog bark) gibi
        sound_name=sound_file.split(".")[0] #Ses dosyasının uzantısız ismi. 9223-2-0-4.wav'ın 9223-2-0-4 ismi gibi
        save_image(sound_folder+"/"+sound_file, main_folder+"/"+sound_label+"/"+sound_name+".png") #Bir üstteki fonksiyon ile resmi kaydedelim.

* Artık resimleri oluşturduk. Drive'daki our_spectrograms resim dosyasının içindeki dosyalara bakarsan resimlerin etiketlerine göre dosyalara kaydedilmiş olduklarını görebilirsin. 

## 2. Önişleme - Notebook
a. Görüntüleri (spectrogramları) sırasıyla okuyarak, grayscale dönüşümü, resizing ve normalizasyon yapın.

b. Görüntüleri ait oldukları etiketlerle birlikte, [görüntü, etiket] formatında bir listeye ekleyin.

c. Bu listeyi kullanarak, X_train, y_train, X_val, y_val, X_test ve y_test veri setlerini oluşturun.

d. Bu veri setlerini bilgisayarınıza kaydedin.


* Artık ses dosyalarıyla bir işimiz kalmadı. Ses dosyalarını renkli resimlere dönüştürdük ve etiketlerine göre birbirlerinden ayırdık.

# Daha önce söylediğimiz gibi; GPU kısıtlamasından dolayı ses dosyalarının tamamını resme dönüştüremedik. Ancak Global AI Hub, proje dökümanında ses dosyalarının tamamının spektrogramlarını bizimle paylaşmış.

"Değerlendirme sadece “Önişleme” ve “Model Hazırlanması ve Eğitilmesi” notebookları üzerinden olacaktır! Sizden 2 farklı notebook beklenmektedir. Spectrogram oluşturma kısmını isteyen arkadaşlarımız yapmayarak, hazırlanmış spectrogramları aşağıdaki linkten indirerek çalışmalarına bunun üzerinden devam edebilirler."

Spectrograms:https://drive.google.com/drive/folders/1xey3vAVNDjWxnSfhuUsf_5dGANZWmCog?usp=sharing

# Bu linkteki "spectrograms" dosyasını kendi drive'ımıza şöyle alıyoruz: 

* 1-) Bu linke tıkladıktan sonra sol sütunda bulunan "Shared With Me(Benimle Paylaşılanlar)" butonuna tıklanır. 

* 2-) En üstte bulunan "spectrograms" klasörüne sağ tıklanır ve "Add Shortcut to Drive(Drive'a Kısayol Ekle)" butonuna tıklanır. 

3-) Ekrana gelen pencereyle istenilen konuma dosya getirilir ve "Add Shortcut(Kısayol Ekle)" butonuna basılır.

Böylece Global AI Hub'ın "spectrograms" klasörü kendi drive'ımıza gelmiş olur. Ben bu klasörü "global_ai_hub" klasörünün içerisine gönderdim.

In [11]:
main_folder="/content/drive/My Drive/global_ai_hub/spectrograms"

In [12]:
# Global AI Hub'ın bize sağladığı spectrograms dosyasının içindeki etiket dosyaları:
!ls "/content/drive/My Drive/global_ai_hub/spectrograms"

0  1  2  3  4  5  6  7	8  9


### Ön işleme aşamasında üç hedef var: 

=> İlk önce renkli resimleri gri hale çevireceğiz. 

=> Resimlerin boyutları birbirinden farklı. CNN'de eğitim yapabilmemiz için resimleri aynı boyuta sokmamız şart. Bunun için resimlerin hepsini aynı boyuta sokacağız. (ben hepsini (100,100) formuna sokmayı tercih ettim)

=> Resimler 0-255 arası değerler alıyor. 100, 200 gibi sayılarla eğitim yapmak bir hesap yükü getiriyor. Bu yüzden bu piksel değerlerini 0(min)-1(max) arasına çekmek(normalizasyon) eğitimi kolaylaştıracaktır.

In [13]:
X=[] #Tüm resimleri tutacak dizi
y=[] #Tüm resimlerin etiketlerini tutacak dizi
# Her etiket dosyası ismi için(0, 1, 2, 3, 4, ..., 8, 9) sayı dönsün: 
for label in range(10):
  #Linux'taki ls komutunda olduğu gibi her etiket dosyasının içine baksın ve içindeki tüm dosyaları versin.
  images=os.listdir(main_folder+"/"+str(label))  
  # İçteki for döngüsü ile sırasıyla tüm resimleri gezsin.
  for image_index in range(len(images)):
    img_gray=cv2.imread(main_folder+"/"+str(label)+"/"+images[image_index],0) #Renkli resmi gri olarak(0:gri, 1:renkli) oku.
    resized_gray = cv2.resize(img_gray, (100,100) , interpolation = cv2.INTER_AREA) #Resmi (100,100) boyutuna getir.
    #Resimlerdeki piksellerdeki en büyük değer 255 olduğu için tüm pikselleri 255'e böl.
    normalized_gray=resized_gray/255 #Bu sayede tüm piksel değerleri 0-1 arasında olacak.
    X.append(normalized_gray) #İşlenmiş resmi X listesine ekle.
    y.append(label) #İşlenmiş resmin etiketini y listesine ekle.

* Tüm resimleri X listesine, etiketlerini de y listesine ekledik.

* Toplam Resim Sayısı=8732

In [14]:
print(len(X))
print(len(y))

8732
8732


* X'i(resimlerin listesini) ve y'yi(etiketlerinin listesini) train, validation ve test için böleceğiz. Ama bir problem var :)

* Resimleri ve etiketleri eklerken dikkat ettiysen 0,0,0,0,0...0,1,1,1,...,1,2,2,2,2,2...8,8,8,8,8...,8,8,9,9,9,9,...,9,9 şeklinde ekledik. Gösterelim:

In [15]:
print(y[0])
print(y[1200])
print(y[1500])
print(y[2500])
print(y[3500])
print(y[4500])
print(y[5500])
print(y[6500])
print(y[7500])
print(y[8500])

0
1
2
3
4
5
6
7
8
9


* Eğer veriyi bu halde bölmeye kalkarsak mesela train kısmında 0'dan 7'e kadar, valid kısmında 7'den 8'e, test kısmında 8'den 9'a kadarki etiketli resimleri almak gibi bir senaryo oluşur. Dolayısıyla train verisiyle eğitilen model 7, 8 ve 9 etiketlerini göremez. Dolayısıyla X ve y listesini(aynı hizada) karıştırmamız lazım ki train, valid ve test bölümünde her bölüm için tüm sınıflar var olabilsin.

* Aşağıdaki fonksiyonu internetten buldum. X'i ve y'yi hizalayıp karıştırıyor, sonra tekrar X ve y olarak bize veriyor. Bu şekilde bu problemden de kurtulmuş olduk.

In [16]:
import random
def shuffle_xy(X,y):
  z = list(zip(X, y))
  random.shuffle(z)
  X, y = zip(*z)
  return X, y

In [17]:
# Örnek:
veri=[1, 2, 3]
etiket=["bir", "iki", "üç"]
karıştırılmış_veri, karıştırılmış_etiket=shuffle_xy(veri, etiket)
print(karıştırılmış_veri)
print(karıştırılmış_etiket)

(2, 3, 1)
('iki', 'üç', 'bir')


In [18]:
X, y=shuffle_xy(X, y) #Resim listesi(X) ve etiket listesi(y)'ni aynı hizada karıştıralım.
X=np.array(X)
y=np.array(y)

* Ön işlenmiş resimleri ve etiketleri birer birer csv dosyasına kaydedip "chapter2.ipynb" notebook'unda aynı şekilde açmamız gerekiyor. Etiket dizisi(y) bir boyutlu olduğu için kaydetmede bir problem olmayacaktır. Ancak X üç boyutlu olduğundan dolayı onu kaydedebilmek için X'in içindeki her iki boyutlu resmi düzleyerek bir boyuta getirmemiz, yani X'i 3 boyutludan 2 boyutlu forma getirmemiz gerekiyor.

In [24]:
print(X.shape)

(8732, 100, 100)


In [19]:
flatten_X= X.reshape(len(X),-1) #Üç boyuttan iki boyuta geçiş
print(flatten_X.shape)

(8732, 10000)


In [20]:
import csv
with open("images.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(flatten_X)

In [22]:
outfile = open('labels.csv','w')
out = csv.writer(outfile)
out.writerows(map(lambda x: [x], y))
outfile.close()